In [1]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer

import spacy
import numpy as np
import string
import os
import requests

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-09-21 00:06:30.116682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#  Preprocessing
if not os.path.exists('./pg64317.txt'):
    url = 'https://www.gutenberg.org/cache/epub/64317/pg64317.txt'
    r = requests.get(url)
    with open('pg64317.txt', 'wb') as f:
        f.write(r.content)

processor = spacy.load('en_core_web_sm')
processed_text = processor(text)

processed_sentences = [sentence.lemma_.split() for sentence in processed_text.sents]
cleaned_sentences = []

for sentence in processed_sentences:
    translator = str.maketrans('', '', string.punctuation)
    cleaned_sentence = [word.translate(translator) for word in sentence if word != '']
    cleaned_sentences.append(cleaned_sentence)


# stemmer = PorterStemmer()
# processed_sentences = [[stemmer.stem(word) for word in sentence] for sentence in processed_sentences]

NameError: name 'text' is not defined

In [ ]:
# Create word2vec model
model = Word2Vec(
    sentences=cleaned_sentences,
    vector_size=300,
    window=5,
    min_count=1,
    workers=4
)

model.save('word2vec.model')

In [ ]:
SENTENCE_1 = "I like your shirt"
SENTENCE_2 = "I love your shirt"

# Tokenize the sentences
tokenized_sentence_1 = SENTENCE_1.split()
tokenized_sentence_2 = SENTENCE_2.split()

# # Stem the tokens
# stemmed_sentence_1 = [stemmer.stem(word) for word in tokenized_sentence_1]
# stemmed_sentence_2 = [stemmer.stem(word) for word in tokenized_sentence_2]

In [ ]:
# Compute sentence vectors
def compute_sentence_vector(sentence : list, model):
    vector = np.zeros(model.wv.vector_size)
    count = 0
    
    for word in sentence:
        if word in model.wv.index_to_key:
            print(word)
            vector += model.wv[word]
            count += 1

    if count != 0:
        vector /= count

    return vector

In [ ]:
model = Word2Vec.load('word2vec.model')

sentence1_vec = compute_sentence_vector(tokenized_sentence_1, model)
sentence2_vec = compute_sentence_vector(tokenized_sentence_2, model)
similarity = cosine_similarity(sentence1_vec.reshape(1, -1), sentence2_vec.reshape(1, -1))[0][0]
print("Cosine Similarity: {:.4f}".format(similarity))


In [ ]:
target_word = 'I'

all_words = [word for word, _ in model.wv.key_to_index.items()]
similarities = [(word, model.wv.similarity(target_word, word)) for word in all_words if word != target_word]

# Sort by similarity
sorted_similarities = sorted(similarities, key=lambda x: x[1])

# Get the least similar words
least_similar_words = sorted_similarities[:5]

# Print the results
for word, similarity in least_similar_words:
    print(f"{word}: {similarity}")
